In [11]:
install.packages("data.table")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [4]:
library(data.table)
library(tictoc)


Attaching package: ‘tictoc’


The following object is masked from ‘package:data.table’:

    shift




In [32]:
library(tictoc)

In [2]:
library(tictoc)
library(data.table)

#  INIZIO TIMER GLOBALE
tic("Totale pipeline (Task1 + Task2 + Task3)")

gene_counts_dt <- fread("bulk_counts_long.csv")
sample_dt <- fread("sample_metadata.csv")

# TASK 1:

tic("Task 1")
treated_samples <- sample_dt[condition == "treated", sample_id] #mi filtra i sample e tutti i samples con condition treated.
gene00_counts <- gene_counts_dt[
    gene %like% "^GENE_00",            # Filtro 'i' (righe)
    .(gene, sample_id, count)         # Seleziono 'j' (colonne) -> ora è una data.table!
] #filtra i geni per 00 per treated sample TASK1

filtered_counts_dt <- gene00_counts[sample_id %in% treated_samples,
        .(total_count_GENE_00 = sum(count)), 
          
          by = sample_id # Raggruppa per (by)
        ]
toc()
print(filtered_counts_dt) #per vedere tabella


# TASK 2:

tic("Task 2")
gene_summary_dt <- gene00_counts[
    sample_id %in% treated_samples
    ][,
      .(mean_count = mean(count),
        median_count = median(count)),
      by = gene] #media e mediana TASK2
toc()
print(head(gene_summary_dt, 5)) #per visualizzare tabella


# TASK 3:
tic("Task 3")
merged_dt <- merge(gene_counts_dt, sample_dt, by = "sample_id")
print(merged_dt) #count
            
final_summary_dt <- merged_dt[,
                 .(mean_count = mean(count)),  # calcolo della media
                     by = .(gene, condition)       
                            ]

final_summary_dt <- final_summary_dt[order(gene)]

toc()

print(head(final_summary_dt, 5)) #media

# FINE TIMER GLOBALE 
      
toc()  # chiude timer totale




#data.frame
library(tictoc)

# ===== INIZIO TIMER GLOBALE =====
tic("Task1 + Task2 + Task3")

gene_counts_df <- read.csv("bulk_counts_long.csv") # Usiamo read.csv al posto di fread
sample_df <- read.csv("sample_metadata.csv")

# TASK 1 
tic("Task 1")

treated_samples <- sample_df[sample_df$condition == "treated", "sample_id"] #Filtro "treated"
gene00_counts <- gene_counts_df[grep("^GENE_00", gene_counts_df$gene), ] #Filtro "GENE_00"

# 3. Filtra le counts per i sample "treated" e calcola la somma totale per sample
filtered_gene00_counts <- gene00_counts[gene00_counts$sample_id %in% treated_samples, ] # Filtra per sample_id

filtered_counts_df <- aggregate(
  count ~ sample_id, 
  data = filtered_gene00_counts,  # Calcola la somma raggruppando per sample_id usando aggregate
  FUN = sum
)

names(filtered_counts_df)[names(filtered_counts_df) == "count"] <- "total_count_GENE_00" # Rinomina la colonna della somma

toc()
print(filtered_counts_df)

#Task 2
tic("Task 2") 

mean_median_func <- function(x) {
  c(mean_count = mean(x), median_count = median(x)) # Funzione per calcolare sia media che mediana
}

gene_summary_df_list <- aggregate(
  count ~ gene, 
  data = filtered_gene00_counts, 
  FUN = mean_median_func
)

gene_summary_df <- data.frame(
  gene = gene_summary_df_list$gene,
  gene_summary_df_list$count
)

colnames(gene_summary_df) <- c("gene", "mean_count", "median_count")

toc()
print(head(gene_summary_df, 5))


#Task 3
tic("Task 3")

merged_df <- merge(gene_counts_df, sample_df, by = "sample_id")
print(head(merged_df))

final_summary_df <- aggregate(
  count ~ gene + condition, 
  data = merged_df, 
  FUN = mean
)

names(final_summary_df)[names(final_summary_df) == "count"] <- "mean_count"

final_summary_df <- final_summary_df[order(final_summary_df$gene), ] #ordina per gene

toc()

print(head(final_summary_df,5))

# ===== FINE TIMER GLOBALE =====
toc() # chiude timer totale





Attaching package: ‘data.table’


The following object is masked from ‘package:tictoc’:

    shift




Task 1: 0.024 sec elapsed
    sample_id total_count_GENE_00
       <char>               <int>
 1:       S01                2017
 2:       S03                1945
 3:       S04                2017
 4:       S05                1949
 5:       S07                2064
 6:       S08                2108
 7:       S11                2021
 8:       S12                1959
 9:       S14                2116
10:       S15                2063
11:       S16                2132
12:       S17                2102
13:       S18                2146
14:       S22                1981
15:       S23                1986
Task 2: 0.006 sec elapsed
        gene mean_count median_count
      <char>      <num>        <num>
1: GENE_0049   7.466667            7
2: GENE_0054  12.333333           12
3: GENE_0018  16.133333           17
4: GENE_0004   5.933333            7
5: GENE_0070   7.933333            7
Key: <sample_id>
       sample_id      gene count condition  batch patient_id timepoint
          <char>    <ch

In [18]:
tasks <- c("Task 1", "Task 2", "Task 3", "Totale Pipeline")
tempo_dt <- c(0.0549, 0.0040, 0.0460, 0.1050)
tempo_df <- c(0.1830, 0.1330, 0.3270, 0.6430)

# 2. Calcolo del Ratio (Vantaggio DT/DF)
# Usiamo i dati grezzi per calcolare il ratio
ratio <- round(tempo_df / tempo_dt, 2)

# 3. Creazione del data.frame
comparison_df <- data.frame(
    Task = tasks,
    'data.table (s)' = tempo_dt,
    'data.frame (s)' = tempo_df,
    'Vantaggio (Ratio DF/DT)' = ratio,
    check.names = FALSE # Mantiene i nomi delle colonne con spazi
)

# 4. Formattazione e Stampa
# Usiamo la funzione format per garantire 4 decimali nei tempi
comparison_df$`data.table (s)` <- format(comparison_df$`data.table (s)`, nsmall = 4)
comparison_df$`data.frame (s)` <- format(comparison_df$`data.frame (s)`, nsmall = 4)

print("--- Risultati Comparazione Tempi data.table vs data.frame ---")
print(comparison_df)

[1] "--- Risultati Comparazione Tempi data.table vs data.frame ---"
             Task data.table (s) data.frame (s) Vantaggio (Ratio DF/DT)
1          Task 1         0.0549         0.1830                    3.33
2          Task 2         0.0040         0.1330                   33.25
3          Task 3         0.0460         0.3270                    7.11
4 Totale Pipeline         0.1050         0.6430                    6.12


In [3]:
#tabella confronto tempi
T_DT1 <- 0.024
T_DT2 <- 0.006
T_DT3 <- 0.002
T_DF1 <- 0.010
T_DF2 <- 0.012
T_DF3 <- 0.033

# Creazione della tabella riassuntiva
risultati_performance <- data.frame(
  Task = c("Task 1",
           "Task 2",
           "Task 3"),
  
  Tempo_data.table_Sec = c(T_DT1, T_DT2, T_DT3),
  Tempo_data.frame_Sec = c(T_DF1, T_DF2, T_DF3)
)

# Aggiungiamo una colonna per il fattore di velocizzazione (Speedup)
risultati_performance$Speedup_DT_vs_DF <- 
  round(risultati_performance$Tempo_data.frame_Sec / risultati_performance$Tempo_data.table_Sec, 1)

# Stampiamo la tabella finale
print(risultati_performance)

    Task Tempo_data.table_Sec Tempo_data.frame_Sec Speedup_DT_vs_DF
1 Task 1                0.024                0.010              0.4
2 Task 2                0.006                0.012              2.0
3 Task 3                0.002                0.033             16.5


In [2]:
# Ti dice la directory di lavoro corrente.
getwd()

[1] "/home/project"